In [7]:
import sys
from pathlib import Path

# Get the project root 
project_root = Path.cwd().parent

# Add project root to sys.path
sys.path.append(str(project_root))

In [8]:
import pandas as pd
import importlib

# Import and reload to get latest changes
import src.preprocessing as preprocessing
importlib.reload(preprocessing)

from src.preprocessing import load_raw_2015

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [9]:
# Step 1: Load raw data
df_raw = load_raw_2015()
print(f"Raw data shape: {df_raw.shape}")
print(f"Columns: {df_raw.shape[1]}")
df_raw.head()

Raw data shape: (441456, 171)
Columns: 171


,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,BPMEDS,BLOODCHO,CHOLCHK,TOLDHI2,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,SEX,MARITAL,EDUCA,VETERAN3,EMPLOY1,CHILDREN,INCOME2,INTERNET,WEIGHT2,HEIGHT3,QLACTLM2,USEEQUIP,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,SMOKE100,SMOKDAY2,USENOW3,ALCDAY5,AVEDRNK2,DRNK3GE5,MAXDRNKS,FRUITJU1,FRUIT1,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,EXRACT11,EXEROFT1,EXERHMM1,EXRACT21,EXEROFT2,EXERHMM2,STRENGTH,LMTJOIN3,ARTHDIS2,ARTHSOCL,JOINPAIN,SEATBELT,FLUSHOT6,FLSHTMY2,IMFVPLAC,PNEUVAC3,HIVTST6,PCDMDECN,TRNSGNDR,QSTVER,QSTLANG,EXACTOT1,EXACTOT2,_CHISPNC,_RFHLTH,_HCVU651,_RFHYPE5,_CHOLCHK,_RFCHOL,_MICHD,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR1,_PRACE1,_MRACE1,_HISPANC,_RACE,_RACEG21,_RACEGR3,_RACE_G1,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,DROCDY3_,_RFBING5,_DRNKWEK,_RFDRHV5,FTJUDA1_,FRUTDA1_,BEANDAY_,GRENDAY_,ORNGDAY_,VEGEDA1_,_MISFRTN,_MISVEGN,_FRTRESP,_VEGRESP,_FRUTSUM,_VEGESUM,_FRTLT1,_VEGLT1,_FRT16,_VEG23,_FRUITEX,_VEGETEX,_TOTINDA,METVL11_,METVL21_,MAXVO2_,FC60_,ACTIN11_,ACTIN21_,PADUR1_,PADUR2_,PAFREQ1_,PAFREQ2_,_MINAC11,_MINAC21,STRFREQ_,PAMISS1_,PAMIN11_,PAMIN21_,PA1MIN_,PAVIG11_,PAVIG21_,PA1VIGM_,_PACAT1,_PAINDX1,_PA150R2,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,5.0,15.0,18,10.0,1,1,2.0,1.0,1.0,1.0,1,1.0,1.0,2,2.0,2,1,2.0,2,1,1.0,1,2,3.0,2,1,4,2.0,8,88.0,3.0,2.0,280.0,510.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,3.0,888.0,NaN,NaN,NaN,305.0,310.0,320.0,310.0,305.0,101.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,888.0,1.0,1.0,1.0,6.0,1.0,1.0,112014.0,1.0,1.0,1.0,b'',NaN,10,1.0,b'',b'',NaN,2,1,2,1,2.0,2.0,2,2,1,1.0,1,1,2,1,1,1,1.0,9,1,63,5,70.0,178.0,12701.0,4018.0,4.0,2,1,2,2,3,1,2,5.400000e-79,1,5.400000e-79,1,1.700000e+01,33.0,6.700000e+01,33.0,17.0,100.0,5.400000e-79,5.400000e-79,1.000000e+00,1.000000e+00,50.0,217.0,2,1,1.0,1.0,5.400000e-79,5.400000e-79,2,NaN,NaN,2469,423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.400000e-79,5.400000e-79,NaN,NaN,NaN,NaN,NaN,NaN,4,2,3,3,2,2,4,2,1.0,1.0,1.0,1,1,NaN,NaN,1.0
1,3.0,88.0,88,NaN,2,1,1.0,4.0,3.0,NaN,1,4.0,2.0,2,2.0,2,2,2.0,2,2,2.0,2,2,3.0,2,2,6,2.0,3,88.0,1.0,1.0,165.0,508.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,3.0,888.0,NaN,NaN,NaN,302.0,305.0,302.0,202.0,202.0,304.0,1.0,64.0,212.0,100.0,69.0,212.0,100.0,888.0,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,2.0,2.0,b'',NaN,10,1.0,b'',b'',NaN,1,2,1,2,1.0,2.0,1,1,3,2.0,1,1,2,1,1,1,1.0,7,1,52,4,68.0,173.0,7484.0,2509.0,3.0,2,1,4,1,1,2,2,5.400000e-79,1,5.400000e-79,1,7.000000e+00,17.0,7.000000e+00,29.0,29.0,13.0,5.400000e-79,5.400000e-79,1.000000e+00,1.000000e+00,24.0,78.0,2,2,1.0,1.0,5.400000e-79,5.400000e-79,1,35.0,5.400000e-79,2876,493,1.0,5.400000e-79,60.0,60.0,2800.0,2800.0,168.0,5.400000e-79,5.400000e-79,5.400000e-79,168.0,5.400000e-79,168.0,5.400000e-79,5.400000e-79,5.400000e-79,2,1,1,2,2,2,2,2,3.0,3.0,4.0,2,2,NaN,NaN,2.0
2,4.0,15.0,88,88.0,1,2,2.0,1.0,3.0,NaN,1,1.0,1.0,7,2.0,1,2,2.0,1,2,1.0,2,2,3.0,2,2,4,2.0,7,88.0,99.0,2.0,158.0,511.0,2.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'',NaN,10,1.0,b'',b'',NaN,2,9,1,1,2.0,NaN,1,1,3,1.0,1,1,2,1,1,1,1.0,11,2,71,6,71.0,180.0,7167.0,2204.0,2.0,1,1,2,9,9,9,9,9.000000e+02,9,9.990000e+04,9,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e+00,4.000000e+00,5.400000e-79,5.400000e-79,NaN,NaN,9,9,1.0,1.0,1.000000e+00,1.000000e+00,9,NaN,NaN,2173,373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,9,9,9,9,9,9,9,9,9.0,9.0,9.0,9,9,9.0,9.0,NaN
3,5.0,30.0,30,30.0,1,2,1.0,1.0,1.0,1.0,1,1.0,1.0,2,2.0,2,2,2.0,1,2,1.0,1,2,3.0,2,1,4,2.0,8,1.0,8.0,2.0,180.0,507.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,NaN,3.0,888.0,NaN,NaN,NaN,555.0,101.0,555.0,301.0,301.0,201.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,888.0,1.0,1.0,1.0,8.0,1.0,1.0,777777.0,5.0,1.0,9.0,b'',NaN,10,1.0,b'',b'',NaN,

## Step 2: Apply Codebook Cleaning

BRFSS uses coded values for missing data (e.g., 7, 9, 77, 99, 777, 999). We need to convert these to proper NaN values so they're recognized as missing data in our analysis.

**Examples:**
- Yes/No questions: 1=Yes, 2=No, 7=Don't know, 9=Refused → Convert 7,9 to NaN
- Frequency questions: 555=Never, 777=Don't know, 999=Refused → Convert 777,999 to NaN, 555 to 0
- Days (0-30): 88=None, 77=Don't know, 99=Refused → Convert 88 to 0, 77,99 to NaN

In [10]:
from src.preprocessing import apply_codebook_cleaning

df_cleaned = apply_codebook_cleaning(df_raw)
print(f"After codebook cleaning: {df_cleaned.shape}")
print(f"\nMissing values before: {df_raw.isna().sum().sum():,}")
print(f"Missing values after: {df_cleaned.isna().sum().sum():,}")
print(f"Increase: {df_cleaned.isna().sum().sum() - df_raw.isna().sum().sum():,}")

After codebook cleaning: (441456, 171)

Missing values before: 9,167,996
Missing values after: 11,144,015
Increase: 1,976,019


## Step 3: Drop Raw Columns When Calculated Versions Exist

BRFSS provides both raw and calculated versions of some variables. For example:
- `WEIGHT2` (raw weight in pounds) → `WTKG3` (calculated weight in kg)
- `HEIGHT3` (raw height) → `HTIN4`, `HTM4`, `_BMI5` (calculated height and BMI)
- `ALCDAY5` (raw alcohol frequency) → `_DRNKWEK` (calculated drinks per week)

**Why drop raw versions?**
- Avoid redundancy and multicollinearity
- Calculated versions are standardized and easier to use
- Reduces dimensionality

Let's check which columns will be dropped:

In [11]:
from src.preprocessing import RAW_TO_CALC, drop_raw_when_calc_exists

# Check which columns will be dropped
print("Raw columns that will be dropped if calculated versions exist:\n")
for raw_col, calc_cols in RAW_TO_CALC.items():
    if raw_col in df_cleaned.columns:
        existing_calc = [c for c in calc_cols if c in df_cleaned.columns]
        if existing_calc:
            print(f"✓ {raw_col} → {existing_calc}")

Raw columns that will be dropped if calculated versions exist:

✓ WEIGHT2 → ['WTKG3', '_BMI5']
✓ HEIGHT3 → ['HTIN4', 'HTM4', '_BMI5']
✓ ALCDAY5 → ['DROCDY3_', '_DRNKWEK', '_RFDRHV5', '_RFBING5']
✓ FRUITJU1 → ['FTJUDA1_']
✓ FRUIT1 → ['FRUTDA1_']
✓ FVBEANS → ['BEANDAY_']
✓ FVGREEN → ['GRENDAY_']
✓ FVORANG → ['ORNGDAY_']
✓ VEGETAB1 → ['VEGEDA1_']
✓ EXEROFT1 → ['PAFREQ1_']
✓ EXERHMM1 → ['PADUR1_']
✓ EXEROFT2 → ['PAFREQ2_']
✓ EXERHMM2 → ['PADUR2_']
✓ STRENGTH → ['STRFREQ_', '_PASTRNG']


In [12]:
# Apply the dropping
df_no_raw = drop_raw_when_calc_exists(df_cleaned)
print(f"\nBefore: {df_cleaned.shape[1]} columns")
print(f"After: {df_no_raw.shape[1]} columns")
print(f"Dropped: {df_cleaned.shape[1] - df_no_raw.shape[1]} columns")


Before: 171 columns
After: 157 columns
Dropped: 14 columns


## Step 4: Identify and Drop High-Missing Columns

Columns with >50% missing values provide little information and can harm model performance. Let's identify them first before deciding to drop.

In [13]:
from src.preprocessing import get_high_missing_columns, drop_columns

# Identify high-missing columns
high_missing_cols = get_high_missing_columns(df_no_raw, threshold=0.5)

print(f"Columns with >50% missing values ({len(high_missing_cols)}):\n")
missing_ratio = df_no_raw[high_missing_cols].isna().mean().sort_values(ascending=False)
for col, ratio in missing_ratio.items():
    print(f"  {col}: {ratio*100:.1f}% missing")

Columns with >50% missing values (17):

  _CHISPNC: 86.0% missing
  JOINPAIN: 70.3% missing
  ARTHDIS2: 70.0% missing
  _PNEUMO2: 69.5% missing
  ARTHSOCL: 69.4% missing
  LMTJOIN3: 69.4% missing
  _FLSHOT6: 68.5% missing
  TRNSGNDR: 62.8% missing
  BPMEDS: 59.7% missing
  FLSHTMY2: 59.5% missing
  SMOKDAY2: 58.4% missing
  IMFVPLAC: 56.8% missing
  PADUR2_: 56.4% missing
  PAFREQ2_: 55.9% missing
  MAXDRNKS: 53.9% missing
  AVEDRNK2: 53.1% missing
  DRNK3GE5: 53.0% missing


In [14]:
# Drop high-missing columns
df_no_high_missing = drop_columns(df_no_raw, high_missing_cols, verbose=True)
print(f"\n✓ Kept {df_no_high_missing.shape[1]} columns")

Dropping 17 columns:
  - _CHISPNC
  - JOINPAIN
  - ARTHDIS2
  - _PNEUMO2
  - ARTHSOCL
  - LMTJOIN3
  - _FLSHOT6
  - TRNSGNDR
  - BPMEDS
  - FLSHTMY2
  - SMOKDAY2
  - IMFVPLAC
  - PADUR2_
  - PAFREQ2_
  - MAXDRNKS
  - AVEDRNK2
  - DRNK3GE5

✓ Kept 140 columns


## Step 5: Drop Questionnaire Metadata Columns

Columns like questionnaire version (`QSTVER`) and interview language (`QSTLANG`) are administrative metadata, not health data. They don't contribute to predicting health outcomes.

In [15]:
from src.preprocessing import drop_questionnaire_metadata_columns

df_no_metadata = drop_questionnaire_metadata_columns(df_no_high_missing)
print(f"Columns remaining: {df_no_metadata.shape[1]}")


Dropping 2 questionnaire metadata columns: ['QSTVER', 'QSTLANG']
Columns remaining: 138


## Step 6: Drop High-Cardinality Text Columns

Free-text columns with many unique values (like "other activity type") are difficult to use in models and typically provide little predictive value.

In [16]:
from src.preprocessing import drop_high_cardinality_text_columns

df_no_text = drop_high_cardinality_text_columns(df_no_metadata, threshold=50)
print(f"Columns remaining: {df_no_text.shape[1]}")


Dropping 2 high-cardinality text columns (>50 unique values):
  - EXACTOT1 (5642 unique values)
  - EXACTOT2 (7623 unique values)
Columns remaining: 136


## Step 7: Fix Boolean Encoding

BRFSS encodes Yes/No questions as 1=Yes, 2=No. For modeling, we need 0=No, 1=Yes (standard binary encoding).

Also, some columns have scientific notation issues (5.4e-79 instead of 0).

In [17]:
from src.preprocessing import fix_boolean_encoding

# Check before
print("Before encoding fix (sample boolean columns):")
bool_sample = [col for col in df_no_text.columns if df_no_text[col].dropna().nunique() == 2][:3]
for col in bool_sample:
    print(f"  {col}: {sorted(df_no_text[col].dropna().unique())}")

df_fixed = fix_boolean_encoding(df_no_text)

# Check after
print("\nAfter encoding fix:")
for col in bool_sample:
    print(f"  {col}: {sorted(df_fixed[col].dropna().unique())}")

# fix target
df_fixed["_MICHD"] = df_fixed["_MICHD"].replace({1: 1, 2: 0})


Before encoding fix (sample boolean columns):
  HLTHPLN1: [np.float64(1.0), np.float64(2.0)]
  MEDCOST: [np.float64(1.0), np.float64(2.0)]
  BLOODCHO: [np.float64(1.0), np.float64(2.0)]

Mapping Yes/No (1=Yes,2=No → 1/0) for 32 columns:
   ['HLTHPLN1', 'MEDCOST', 'TOLDHI2', 'CVDINFR4', 'CVDCRHD4', 'CVDSTRK3', 'ASTHMA3', 'CHCSCNCR', 'CHCOCNCR', 'CHCCOPD1', 'HAVARTH3', 'ADDEPEV2', 'CHCKIDNY', 'QLACTLM2', 'USEEQUIP', 'FLUSHOT6', 'PNEUVAC3', 'DRNKANY5', '_FRTLT1', '_VEGLT1', '_TOTINDA', '_PAINDX1', '_PA30021', '_PASTRNG', '_PAREC1', '_PASTAE1', '_LMTACT1', '_LMTWRK1', '_LMTSCL1', '_RFSEAT2', '_RFSEAT3', 'PAMISS1_']

Mapping risk flags (1=No,2=Yes → 0/1) for 6 columns:
   ['_RFHYPE5', '_RFCHOL', '_RFBMI5', '_RFSMOK3', '_RFBING5', '_RFDRHV5']

✓ Fixed encoding for 60 boolean-like columns

After encoding fix:
  HLTHPLN1: [np.float64(0.0), np.float64(1.0)]
  MEDCOST: [np.float64(0.0), np.float64(1.0)]
  BLOODCHO: [np.float64(1.0), np.float64(2.0)]


## Step 8: Drop Rows with Missing Target Variable

For supervised learning, we need the target variable (`_MICHD` - cardiovascular disease indicator). Rows without this value cannot be used for training or evaluation.

In [18]:
# Define target variable
TARGET_COL = '_MICHD'

# Check missing target values
print(f"Before dropping missing target:")
print(f"  Total rows: {df_fixed.shape[0]:,}")
print(f"  Rows with missing target: {df_fixed[TARGET_COL].isna().sum():,}")
print(f"  Percentage missing: {df_fixed[TARGET_COL].isna().mean()*100:.2f}%")

# Drop rows with missing target
df_no_missing_target = df_fixed.dropna(subset=[TARGET_COL])

print(f"\nAfter dropping missing target:")
print(f"  Total rows: {df_no_missing_target.shape[0]:,}")
print(f"  Rows dropped: {df_fixed.shape[0] - df_no_missing_target.shape[0]:,}")

Before dropping missing target:
  Total rows: 441,456
  Rows with missing target: 3,942
  Percentage missing: 0.89%

After dropping missing target:
  Total rows: 437,514
  Rows dropped: 3,942


## Step 9: Feature Grouping and Cleanup

Apply lightweight feature consolidation: scale MET/VO₂/PA frequency values, drop redundant or noisy raw variables, combine physical-difficulty indicators, and retain only compact BMI, SES, smoking, alcohol, and diet feature blocks for analysis.

In [19]:
from src.preprocessing import feature_grouping

df_final = feature_grouping(df_no_missing_target)

# Update df_prepared
df_prepared = df_final


================ Feature grouping / cleaning report ================

✓ Scaled MET variables (÷10, fix 5.4e-79): ['METVL11_', 'METVL21_']
✓ Scaled VO2 / functional capacity (÷100): ['MAXVO2_', 'FC60_']
✓ Scaled PA frequency (÷1000): ['PAFREQ1_']
✓ Combined difficulty indicators ['BLIND', 'DECIDE', 'DIFFWALK', 'DIFFDRES', 'DIFFALON'] → 'PHYSICAL_DIFFICULTY'
✓ Dropped admin / duplicate / out-of-scope columns: ['SMOKE100', 'USENOW3', 'EXERANY2', 'SEATBELT', 'HIVTST6', 'PCDMDECN', '_RFHLTH', '_HCVU651', '_LTASTH1', '_CASTHM1', '_DRDXAR1', '_PRACE1', '_MRACE1', '_HISPANC', '_RACE', '_RACEG21', '_RACE_G1', '_AGEG5YR', '_AGE65YR', '_AGE_G', 'VETERAN3', 'INTERNET', 'BLOODCHO', 'PA1MIN_', 'PA1VIGM_', '_AIDTST3']
✓ Dropped PA/HIV technical columns: ['PA1MIN_', 'PA1VIGM_', '_AIDTST3']
✓ Dropped raw body-size columns: ['HTIN4', 'HTM4', 'WTKG3', '_BMI5'] (keep _BMI5CAT and _RFBMI5)
✓ Dropped detailed alcohol volume columns: ['DROCDY3_', '_DRNKWEK'] (keep DRNKANY5, _RFBING5, _RFDRHV5)
✓ Dropped det

## Step 10: Classify Variables and Save Final Dataset

Now classify columns into numeric, categorical, and boolean types, then save the final preprocessed dataset.

In [20]:
from src.preprocessing import (save_preprocessed_csv, classify_variables)

# Classify variables
numeric_cols, categorical_cols, boolean_cols = classify_variables(df_prepared)

# Save
save_path = save_preprocessed_csv(df_prepared, "clean_2015.csv")

print(f"\n✓ Final preprocessed dataset: {df_prepared.shape}")
print(f"✓ Saved to: {save_path}")


=== Variable Classification ===
Numeric: 24
Categorical: 26
Boolean: 36
Saved cleaned dataset to: E:\DataPrepPrj\data\processed\clean_2015.csv

✓ Final preprocessed dataset: (437514, 86)
✓ Saved to: E:\DataPrepPrj\data\processed\clean_2015.csv


In [21]:
# Summary of final preprocessed data
print(f"\n{'='*50}")
print(f"FINAL PREPROCESSED DATASET SUMMARY")
print(f"{'='*50}")
print(f"Total columns: {df_prepared.shape[1]}")
print(f"Total rows: {df_prepared.shape[0]}")
print(f"\nVariable breakdown:")
print(f"  - Numeric: {len(numeric_cols)}")
print(f"  - Categorical: {len(categorical_cols)}")
print(f"  - Boolean: {len(boolean_cols)}")
print(f"\nMissing values: {df_prepared.isna().sum().sum():,} ({df_prepared.isna().sum().sum() / df_prepared.size * 100:.2f}%)")


FINAL PREPROCESSED DATASET SUMMARY
Total columns: 86
Total rows: 437514

Variable breakdown:
  - Numeric: 24
  - Categorical: 26
  - Boolean: 36

Missing values: 3,975,704 (10.57%)


In [22]:
df_prepared.head(10)

,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,HLTHPLN1,PERSDOC2,MEDCOST,CHECKUP1,BPHIGH4,CHOLCHK,TOLDHI2,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,CHCSCNCR,CHCOCNCR,CHCCOPD1,HAVARTH3,ADDEPEV2,CHCKIDNY,DIABETE3,SEX,MARITAL,EDUCA,EMPLOY1,CHILDREN,INCOME2,QLACTLM2,USEEQUIP,EXRACT11,EXRACT21,FLUSHOT6,PNEUVAC3,_RFHYPE5,_CHOLCHK,_RFCHOL,_MICHD,_ASTHMS1,_RACEGR3,_AGE80,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG,_SMOKER3,_RFSMOK3,DRNKANY5,_RFBING5,_RFDRHV5,_FRUTSUM,_VEGESUM,_FRTLT1,_VEGLT1,_TOTINDA,METVL11_,METVL21_,MAXVO2_,FC60_,ACTIN11_,ACTIN21_,PADUR1_,PAFREQ1_,_MINAC11,_MINAC21,STRFREQ_,PAMISS1_,PAMIN11_,PAMIN21_,PAVIG11_,PAVIG21_,_PACAT1,_PAINDX1,_PA150R2,_PA300R2,_PA30021,_PASTRNG,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,PHYSICAL_DIFFICULTY
0,5.0,15.0,18.0,10.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,3.0,2,1.0,4.0,8.0,0.0,3.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,63,4.0,1.0,1.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,50.0,217.0,0.0,1.0,0.0,NaN,NaN,24.69,4.23,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,4.0,0.0,3.0,3.0,0.0,0.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,7.0
1,3.0,0.0,0.0,NaN,0.0,1.0,1.0,4.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2,2.0,6.0,3.0,0.0,1.0,1.0,0.0,64.0,69.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,1.0,52,3.0,1.0,1.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,24.0,78.0,0.0,0.0,1.0,3.5,0.0,28.76,4.93,1.0,0.0,60.0,2.8,168.0,0.0,0.0,0.0,168.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,0.0,0.0,8.0
3,5.0,30.0,30.0,30.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,2,1.0,4.0,8.0,1.0,8.0,1.0,0.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,3.0,1.0,63,3.0,1.0,2.0,2.0,5.0,4.0,0.0,0.0,0.0,0.0,100.0,20.0,1.0,0.0,0.0,NaN,NaN,24.69,4.23,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,4.0,0.0,3.0,3.0,0.0,0.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,6.0
4,5.0,20.0,0.0,30.0,1.0,1.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2,1.0,5.0,8.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,3.0,1.0,61,2.0,0.0,1.0,3.0,NaN,4.0,0.0,0.0,0.0,0.0,NaN,200.0,NaN,1.0,0.0,NaN,NaN,25.43,4.36,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,4.0,0.0,3.0,3.0,0.0,0.0,4.0,0.0,1.0,1.0,1.0,1.0,1.0,10.0
5,2.0,0.0,0.0,NaN,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2,3.0,3.0,2.0,0.0,6.0,1.0,0.0,18.0,73.0,1.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,73,3.0,1.0,1.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,200.0,600.0,1.0,1.0,1.0,5.0,3.3,20.99,3.60,2.0,1.0,60.0,1.0,60.0,210.0,0.0,0.0,120.0,210.0,60.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,10.0
6,2.0,0.0,3.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2,3.0,5.0,7.0,0.0,4.0,0.0,0.0,18.0,64.0,0.0,1.0,1.0,1.0,1.0,0.0,3.0,4.0,70,2.0,0.0,1.0,3.0,2.0,4.0,0.0,1.0,0.0,0.0,129.0,114.0,1.0,1.0,1.0,5.0,3.5,22.10,3.79,2.0,1.0,60.0,2.0,120.0,105.0,0.0,0.0,240.0,105.0,120.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,3.0,4.0,1.0,1.0,10.0
8,5.0,NaN,0.0,NaN,1.0,1.0,0.0,1.0,3.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2,3.0,3.0,5.0,0.0,NaN,1.0,1.0,98.0,88.0,1.0,1.0,0.0,NaN,NaN,0.0,3.0,1.0,80,1.0,0.0,1.0,1.0,NaN,4.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,4.5,0.0,16.18,2.77,2.0,0.0,60.0,3.0,180.0,0.0,NaN,0.0,360.0,0.0,180.0,0.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,0.0,1.0,0.0,1.0,1.0,8.0
9,2.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1,1.0,6.0,7.0,0.0,8.0,0.0,0.0,64.0,88.0,0.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,68,3.0,1.0,1.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,114.0,314.0,1.0,1.0,1.0,3.5,0.0,22.60,3.87,1.0,0.0,50.0,6.0,300.0,0.0,0.0,0.0,300.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,1.0,0.0,10.0
10,3.0,14.0,0.0,14.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2,1.0,6.0,7.0,0.0,7.0,0.0,0.0,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,3.0,1.0,62,4.0,1.0,1.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,43.0,96.0,0.0,0.0,0.0,NaN,NaN,25.06,4.30,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,4.0,0.0,3.0,3.0,0.0,0.0,4.0,0.0,3.0,3.0,4.0,1.0,1.0,10.0
11,5.0,0.0,0.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,